## Massaging and Rewieghing Method

In [36]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, roc_auc_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
import fair

In [18]:
train= pd.read_csv('Train_v2.csv')

In [19]:
#Load data by converting to categorical variables
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def load_data(data, s_attr):
    X= data.drop(['bank_account','uniqueid'],axis=1)
    Y= data['bank_account']
    Y=pd.Series([0 if x=='No' else 1 for x in Y])
    categorical_feature_mask = X.dtypes==object
    categorical_cols = X.columns[categorical_feature_mask].tolist()
    X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
    Z = X.loc[:, s_attr]
#     X= X.drop([s_attr],axis=1)
    return X, Y, Z

In [17]:
def DP_accuracy (y_hat,A):
    A , y_hat = np.array(A).reshape(-1) , y_hat.reshape(-1)
    sum_A0 = 0
    sum_A1 = 0
    n_A1 = np.count_nonzero(A)
    n_A0 = len(A) - np.count_nonzero(A)
    for i in range (len(y_hat)):
        sum_A0 += y_hat[i]*(1-A[i])
        sum_A1 += y_hat[i]*A[i]
    accuracy= abs(sum_A0/n_A0 - sum_A1/n_A1)
    return accuracy

def p_rule(y_pred, z_values):
    y_z_1 =np.array([i for i,j in zip(y_pred,z_values) if i==1 and j==1])
    y_z_0 = np.array([i for i,j in zip(y_pred,z_values) if i==1 and j==0])
    odds = y_z_1.sum() / y_z_0.sum()
    return  np.min([odds, 1/odds]) 

def split(y_pred,y_test,z_test): 
    pred0 =np.array([i for i,j,k in zip(y_pred,y_test,z_test) if k==0])
    pred1 = np.array([i for i,j,k in zip(y_pred,y_test,z_test) if k==1])
    y_test0 =np.array([j for i,j,k in zip(y_pred,y_test,z_test) if k==0])
    y_test1 = np.array([j for i,j,k in zip(y_pred,y_test,z_test) if k==1])
    return [pred0,y_test0,pred1,y_test1]

def classreport(pred,y_test):
    
    fp = sum(np.logical_and(y_test == 0, pred == 1)) 
    fn = sum(np.logical_and(y_test == 1, pred == 0)) 
    tp = sum(np.logical_and(y_test == 1, pred == 1)) 
    tn = sum(np.logical_and(y_test== 0, pred == 0))
    
    fpr = float(fp) / float(fp + tn)
    fnr = float(fn) / float(fn + tp)
    tpr = float(tp) / float(tp + fn)
    tnr = float(tn) / float(tn + fp)
    npv = float(tn) / float(tn + fn)
    ppv = float(tp) / float(tp + fp)
    acc = (tp + tn) / (tp + tn + fp + fn)
    di= {'False Positive': fp,
         'False Negative': fn,
         'True Positive': tp,
         'True Negative': tn,
         'False Positive rate': fpr,
         'False Negative rate': fnr,
         'True Positive rate': tpr,
         'True Negative rate': tnr,
         'npv': npv,
         'ppv': ppv,
        }
    return di

def Get_fair_metric(model,y_pred,y_test,Z_test):
    pre1,y_test1 = split(y_pred,y_test,Z_test)[0],split(y_pred,y_test,Z_test)[1]
    pre2,y_test2 = split(y_pred,y_test,Z_test)[2],split(y_pred,y_test,Z_test)[3]
    a=classreport(pre1,y_test1)
    b=classreport(pre2,y_test2)
    dp= DP_accuracy (y_pred,Z_test)
    dI=p_rule(y_pred, Z_test)
    aoc = roc_auc_score(y_test, y_pred)
    Equalised_opportunity= a['True Positive rate']- b['True Positive rate']
    Equalised_odds = a['False Positive rate']- b['False Positive rate']
    Treatmentequal= (b['False Positive']/b['False Negative'])/(a['False Positive']/a['False Negative'])
    Predictiveequal= a['True Negative rate']- b['True Negative rate']
    condUseAcc= (a['ppv']+a['npv'])- (b['ppv']+b['npv'])
    OVerUseAcc= (a['True Positive rate']+a['True Negative rate'])- (b['True Positive rate']+b['True Negative rate'])
    D={'model': model,'Demographic Parity':dp, 'Disparate Impact':dI, 'Equalised_opportunity': Equalised_opportunity,
      'Equalised_odds': Equalised_odds, 'Treatment equality': Treatmentequal,
       'Predictive equality' : Predictiveequal, 'Conditional Use Accuracy': condUseAcc, 
      'Overall Use Accuracy': OVerUseAcc, 'ROC Accuracy': aoc}
    return D

In [20]:
#converting the age variable to categorical
def age_cat(age):
    if age >= 15 and age <=65:
        return "15-65"
    else:
        return "above 65"
train['age_cat'] = train['age_of_respondent'].apply(age_cat)

## 1. Massaging the dataset


### Ranking the dataset

In [21]:
#This function ranks the dataset using Naive-Bayes ranker
#input: data
#output: dataset with additional column 'Rank'
def rank(data):
    datcopy=data.drop(['bank_account'],axis=1)
    W=[]
    pos= data.loc[data['bank_account'] == 'Yes']
    neg= data.loc[data['bank_account'] == 'No']
    Ppos= len(pos)/len(data)
    Pneg= 1-Ppos
    for i in range (len(data)):
        W1=[]
        W2=[]
        for j in datcopy.columns:
            attr=(data.loc[i])[j]
            P= len(pos.loc[pos[j] == attr])
            n= len(neg.loc[neg[j] == attr])
            W1.append(P/len(data))
            W2.append(n/len(data))
        po = (np.prod(W1))*Ppos
        ne = (np.prod(W2))*Pneg
        Soln= po/(po+ne)
        W.append(Soln)
    data['Rank']= W
    return data
    

In [7]:
Ranked=rank(train)

In [8]:
trains = train.drop(['uniqueid', 'household_size', 'age_of_respondent','bank_account','Rank'], axis=1)

$$M= \dfrac{bias(X) \times |X_{a}| \times |X_{b}|}{|X|}$$

In [9]:
M= (0.082901* 13877* 9647)/len(train)

In [10]:
m=round(M)

### Promoting and demoting candidates

In [16]:
def generate_data(Ranked, sen_attr_col, unpro_atr):
    # splitting datapoints into promotion and demotion 
    RamF=Ranked.loc[Ranked[sen_attr_col] == unpro_atr]
    # promotion
    pr= RamF.loc[RamF['bank_account'] == 'No']
    df3= RamF.loc[RamF['bank_account'] == 'Yes']
    RamM=Ranked.loc[Ranked[sen_attr_col] != unpro_atr]
    # demotion
    dem= RamM.loc[RamM['bank_account'] == 'Yes']
    df4= RamM.loc[RamM['bank_account'] == 'No']
    #sort values based on rank
    ndem=dem.sort_values(by=['Rank'])
    npr=pr.sort_values(by=['Rank'], ascending=False)
    # promote top m candidates
    c1 =  npr.iloc[0:m]
    c1['bank_account'] = 'Yes'
    c2=npr.iloc[m:]
    Df1=pd.concat([c1,c2])
    # demote top m candidates
    d1 = ndem.iloc[0:m]
    d1['bank_account'] = 'No'
    d2 = ndem.iloc[471:]
    df2= pd.concat([d1,d2])
    #concatenate to obtain new dataset
    New_df= pd.concat([Df1, df2, df3, df4])
    return New_df

In [15]:
New_df = generate_data(Ranked, 'gender_of_respondent', 0)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [17]:
from sklearn.preprocessing import LabelEncoder

In [22]:
X, Y, Z = load_data(New_df, 'gender_of_respondent')
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, Y, Z, test_size=0.5, 
                                                                     stratify=Y, random_state=7)

In [26]:
# def Evaluation_Generator(data, new_data, model, sen_attr_col):
#     datastream= [data,new_data]
#     for i,datum in enumerate(datastream):
#         X, Y, Z = load_data(datum, sen_attr_col)
#         X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, Y, Z, test_size=0.5, 
#                                                                      stratify=Y, random_state=7)
#         model.fit(X_train, y_train)
#         predictions = model.predict(X_test)
#         accuracy= accuracy_score(y_test, predictions)
#         aoc = roc_auc_score(y_test, predictions)
#         DPaccuracy = DP_accuracy (predictions,Z_test)
#         prule = p_rule(predictions,Z_test)
#         if i ==0:
#             l={'model':model, 'Type':'Original Data', 'accuracy':accuracy,
#               'Dp':DPaccuracy,'prule':prule,' ROC_accuracy': aoc}
#             H.append(l)
#         else:
#             l={'model':model, 'Type':'Massaged Data', 'accuracy':accuracy,
#               'Dp':DPaccuracy,'prule':prule,' ROC_accuracy': aoc}
#             H.append(l)        
#     return H

In [31]:
#generating fairness metric values for method
import xgboost as xgb
H=[]
models=[LogisticRegression(), GaussianNB(),
        RandomForestClassifier(n_estimators=100, oob_score=True, random_state=12),
        DecisionTreeClassifier(random_state = 21),MLPClassifier(),xgb.XGBClassifier()]
for model in models:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    D=fair.Get_fair_metric(model,predictions,y_test,Z_test)
    H.append(D)

/home/students/snap/jupyter/common/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars


In [124]:
#drop Rank for next value
train=train.drop(['Rank'],axis=1)

## 2. Reweighing

Assigning weights to datapoints based on 
\begin{equation}
    W(x)= 
    \begin{cases}
    1.31 \ \ \ \ \text{if} \ \ \ X(gender)=Female \ \ \text{and}\ \ X(class)= \text{'Yes'} \\
    0.743 \ \ \ \text{if} \ \ \ X(gender)=Male \ \ \text{and}\ \ X(class)= \text{'Yes'} \\
    0.96 \ \ \ \ \text{if} \ \ \ X(gender)=Female \ \ \text{and}\ \ X(class)= \text{'No'} \\
    1.06 \ \ \ \ \text{if} \ \ \ X(gender)=Male \ \ \text{and}\ \ X(class)= \text{'No'} 
    \end{cases}
\end{equation}

In [40]:
#W1
h=train.loc[train['gender_of_respondent']== 'Female']
W1=h.loc[h['bank_account']=='Yes']
W1['Weight']=1.31

In [41]:
#W2
h=train.loc[train['gender_of_respondent']== 'Male']
W2=h.loc[h['bank_account']=='Yes']
W2['Weight']=0.743

In [42]:
#W3
h=train.loc[train['gender_of_respondent']== 'Female']
W3=h.loc[h['bank_account']=='No']
W3['Weight']=0.92

In [43]:
#W4
h=train.loc[train['gender_of_respondent']== 'Male']
W4=h.loc[h['bank_account']=='No']
W4['Weight']=1.06

In [44]:
#concatenate
WW=pd.concat([W1,W2])
OW=pd.concat([WW,W3])
Final=pd.concat([OW,W4])

In [45]:
#Visualize
Final

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,age_cat,Weight
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed,15-65,1.31
17,Kenya,2018,uniqueid_18,Yes,Rural,Yes,6,27,Female,Head of Household,Married/Living together,Secondary education,Self employed,15-65,1.31
32,Kenya,2018,uniqueid_33,Yes,Urban,Yes,4,62,Female,Spouse,Married/Living together,Primary education,Formally employed Government,15-65,1.31
54,Kenya,2018,uniqueid_55,Yes,Urban,Yes,1,22,Female,Head of Household,Single/Never Married,Secondary education,Self employed,15-65,1.31
63,Kenya,2018,uniqueid_64,Yes,Urban,Yes,1,25,Female,Head of Household,Single/Never Married,Tertiary education,Remittance Dependent,15-65,1.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23506,Uganda,2018,uniqueid_2096,No,Rural,No,7,22,Male,Child,Single/Never Married,Primary education,Self employed,15-65,1.06
23515,Uganda,2018,uniqueid_2108,No,Rural,No,6,16,Male,Parent,Single/Never Married,Primary education,Other Income,15-65,1.06
23516,Uganda,2018,uniqueid_2109,No,Urban,Yes,3,35,Male,Head of Household,Married/Living together,Primary education,Self employed,15-65,1.06
23517,Uganda,2018,uniqueid_2110,No,Urban,Yes,9,16,Male,Parent,Single/Never Married,Primary education,Other Income,15-65,1.06


In [46]:
#Load data again
X, Y, Z = load_data(Final, 'gender_of_respondent')

In [47]:
#Split data again
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, Y, Z, test_size=0.3, 
                                                                     stratify=Y, random_state=7)

In [48]:
#Select weights from dataframe
weights=X_train['Weight']

In [49]:
#drop weights from X_train, X_test beacuse the weighs are only necessary during the fitting
# and are not to be part of the prediction process
X_train, X_test= X_train.drop(['Weight'],axis=1), X_test.drop(['Weight'],axis=1)

In [50]:
#generate fairness metric values for all methods
H=[]
models= [LogisticRegression(random_state=13), 
         GaussianNB(),RandomForestClassifier(random_state=0),
         DecisionTreeClassifier(random_state=0), xgb.XGBClassifier()]
for model in models:
    clf = model
    clf.fit(X_train, y_train, sample_weight=weights)
    prediction = clf.predict(X_test)
    D=fair.Get_fair_metric(model,prediction,y_test,Z_test)
    H.append(D)

In [51]:
#visualize
pd.DataFrame(H)

,model,Demographic Parity,Disparate Impact,Equalised_opportunity,Equalised_odds,Treatment equality,Predictive equality,Conditional Use Accuracy,Overall Use Accuracy,ROC Accuracy
0,"LogisticRegression(C=1.0, class_weight=None, d...",0.009842,0.871951,-0.002880,0.007392,0.320833,-0.007392,-0.181527,-0.010272,0.588268
1,"GaussianNB(priors=None, var_smoothing=1e-09)",0.066919,0.988042,-0.054781,-0.025978,0.669645,0.025978,-0.102528,-0.028803,0.710004
2,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.062388,0.792593,-0.098272,-0.021452,0.837119,0.021452,-0.095104,-0.076820,0.670814
3,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",0.056658,1.000000,-0.069758,-0.020969,0.666613,0.020969,-0.109263,-0.048789,0.663800
4,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.020651,0.949580,-0.000346,0.008162,0.329958,-0.008162,-0.152152,-0.008507,0.654947
